# In honor of September 5

This notebook walks through my analysis of the historical games between the University of Washington and the University of Michigan in honor of the Sept 5 game cancelled due to the Coronavirus pandemic.

The data used in this analysis comes from:  
* [College Football Data API](https://api.collegefootballdata.com/api/docs/?url=/api-docs.json)

-------------
###  Setup

In [17]:
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize

In [2]:
seasons = np.arange(1889,2020)

In [3]:
game_data = []

for season in seasons:
    parameters = {"year": season, "seasonType": "regular", "team": "Washington"}
    response = requests.get("https://api.collegefootballdata.com/games", params=parameters)
    games = response.json()
    for game in games:
        if (game["home_team"] == "Michigan" or game["away_team"] == "Michigan"):
            game_data.append(game)
    parameters = {"year": season, "seasonType": "postseason", "team": "Washington"}
    response = requests.get("https://api.collegefootballdata.com/games", params=parameters)
    games = response.json()
    for game in games:
        if (game["home_team"] == "Michigan" or game["away_team"] == "Michigan"):
            game_data.append(game)

In [4]:
game_data

[{'id': 31481,
  'season': 1953,
  'week': 2,
  'season_type': 'regular',
  'start_date': '1953-09-26T00:00:00.000Z',
  'start_time_tbd': None,
  'neutral_site': False,
  'conference_game': None,
  'attendance': None,
  'venue_id': None,
  'venue': None,
  'home_id': 130,
  'home_team': 'Michigan',
  'home_conference': 'Big Ten',
  'home_points': 50,
  'home_line_scores': [],
  'home_post_win_prob': None,
  'away_id': 264,
  'away_team': 'Washington',
  'away_conference': 'Pacific',
  'away_points': 0,
  'away_line_scores': [],
  'away_post_win_prob': None,
  'excitement_index': None},
 {'id': 32056,
  'season': 1954,
  'week': 2,
  'season_type': 'regular',
  'start_date': '1954-09-25T00:00:00.000Z',
  'start_time_tbd': None,
  'neutral_site': False,
  'conference_game': None,
  'attendance': None,
  'venue_id': None,
  'venue': None,
  'home_id': 264,
  'home_team': 'Washington',
  'home_conference': 'Pacific',
  'home_points': 0,
  'home_line_scores': [],
  'home_post_win_prob': Non

In [5]:
len(game_data)

12

In [6]:
game_df = json_normalize(game_data)
game_df.head()

C:\Users\wwtar\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,id,season,week,season_type,start_date,start_time_tbd,neutral_site,conference_game,attendance,venue_id,...,home_points,home_line_scores,home_post_win_prob,away_id,away_team,away_conference,away_points,away_line_scores,away_post_win_prob,excitement_index
0,31481,1953,2,regular,1953-09-26T00:00:00.000Z,None,False,None,NaN,NaN,...,50,[],None,264,Washington,Pacific,0,[],None,None
1,32056,1954,2,regular,1954-09-25T00:00:00.000Z,None,False,None,NaN,NaN,...,0,[],None,130,Michigan,Big Ten,14,[],None,None
2,41264,1969,3,regular,1969-09-27T00:00:00.000Z,None,False,None,NaN,NaN,...,45,[],None,264,Washington,Pac-8,7,[],None,None
3,41951,1970,5,regular,1970-09-26T00:00:00.000Z,None,False,None,NaN,NaN,...,3,[],None,130,Michigan,Big Ten,17,[],None,None
4,48018,1977,1,postseason,1978-01-02T00:00:00.000Z,None,True,None,NaN,NaN,...,27,[],None,130,Michigan,Big Ten,20,[],None,None


In [7]:
season = []
team = []
points = []

for index, row in game_df.iterrows(): 
    season.append(row["season"])
    team.append(row["home_team"])
    points.append(row["home_points"])
    season.append(row["season"])
    team.append(row["away_team"])
    points.append(row["away_points"])

games = pd.DataFrame()
games["season"] = season
games["team"] = team
games["points"] = points

In [8]:
games.head()

,season,team,points
0,1953,Michigan,50
1,1953,Washington,0
2,1954,Washington,0
3,1954,Michigan,14
4,1969,Michigan,45


In [9]:
games.loc[games.team == "Michigan", "points"] *= -1

In [10]:
season = []
wa_points = []
mich_points = []

for index, row in game_df.iterrows(): 
    season.append(row["season"])
    if(row["home_team"] == "Washington"):
        wa_points.append(row["home_points"])
        mich_points.append(row["away_points"] * -1)
    else:
        wa_points.append(row["away_points"])
        mich_points.append(row["home_points"] * -1)

games = pd.DataFrame()
games["season"] = season
games["wa_points"] = wa_points
games["mich_points"] = mich_points

In [11]:
games

,season,wa_points,mich_points
0,1953,0,-50
1,1954,0,-14
2,1969,7,-45
3,1970,3,-17
4,1977,27,-20
5,1980,6,-23
6,1983,25,-24
7,1984,20,-11
8,1991,34,-14
9,1992,31,-38


In [12]:
games["season"] = games["season"].astype('str') + " season"

In [13]:
years = ["2002 season", "2001 season", "1992 season", "1991 season", "1984 season", "1983 season", "1980 season", "1977 season", "1970 season", "1969 season", "1954 season", "1953 season"]

In [18]:
sns.set_style("whitegrid")

fig = plt.figure(figsize=(14,8))
bar_plot = sns.barplot(x="mich_points", y="season", data=games, order=years, lw=0, color="#00274C", label="Michigan")
bar_plot = sns.barplot(x="wa_points", y="season", data=games, order=years, lw=0, color="#4b2e83", label="Washington")

y_labels = ["2002", "2001", "1992", "1991", "1984", "1983", "1980", "1977", "1970", "1969", "1954", "1953"]
x_labels = ["60", "40", "20", "0", "20", "40", "60"]


bar_plot.set_xlabel("Points", fontsize=15)
bar_plot.set_ylabel("Season", fontsize=15)
bar_plot.set_title("UW vs Michigan over the years",fontsize=20)
bar_plot.set_yticklabels(y_labels, fontsize=15)
bar_plot.set(xlim=(-59,59))
bar_plot.set_xticklabels(x_labels, fontsize=15)

leg = plt.legend(prop={'size': 15})

plt.savefig("in_honor_of_Sept_5.jpg")
plt.show()